<a href="https://colab.research.google.com/github/adikahnf/learn-llm-based-tools/blob/main/Gemini_Part_2_H8_Revamped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Konfigurasi Parameter

In [1]:
!pip install -q -U "google-genai>=1.0.0"

### Import Necessary Libraries

In [2]:
from google import genai
from google.genai import types
from IPython.display import Markdown
from google.colab import userdata

In [9]:
GEMINI = userdata.get('GOOGLE_API_KEY')

# masukan API key kalian
client = genai.Client(api_key=GEMINI)

system_instruction='Kamu adalah pakar AI, Bicaralah seperti layaknya seorang Pakar'

chat_config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    #max_output_tokens=300,
    temperature=0,
    top_p=0.95,
    top_k=20,
)

# temperature, top_p, top_k is relate each others
# temperature -> "Creativity dial" : lower = safe & factual, higher = wild & imaginative
# top_k      -> "Take Top K"      : keep only the K most likely tokens (fixed count finalists)
# top_p      -> "Probability Pool": keep just enough tokens until their combined probability ≥ p

model_id = 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",
)

print(response.text)

Baik, mari kita definisikan Kecerdasan Buatan (Artificial Intelligence - AI) dari perspektif seorang pakar.

**Kecerdasan Buatan (AI) adalah cabang ilmu komputer yang berfokus pada penciptaan mesin yang dapat meniru dan bahkan melampaui kemampuan kognitif manusia.** Ini bukan sekadar otomatisasi tugas, melainkan upaya untuk membangun sistem yang mampu melakukan fungsi-fungsi yang secara tradisional membutuhkan kecerdasan manusia, seperti:

1.  **Pemikiran dan Penalaran:** Kemampuan untuk memproses informasi, menarik kesimpulan logis, dan membuat keputusan.
2.  **Pembelajaran:** Kemampuan untuk memperoleh pengetahuan dan keterampilan dari data atau pengalaman, dan meningkatkan kinerja seiring waktu tanpa pemrograman eksplisit untuk setiap skenario.
3.  **Pemecahan Masalah:** Kemampuan untuk mengidentifikasi masalah, menganalisis opsi, dan menemukan solusi yang efektif.
4.  **Persepsi:** Kemampuan untuk menafsirkan dan memahami informasi sensorik (visual, audio, sentuhan) dari lingkungan

Anda dapat menggunakan `system_instruction`, saat Anda menginisialisasi model AI. Anda dapat memberinya instruksi tentang cara merespons, seperti menetapkan persona ("Anda adalah seorang Data Scientist") atau memberi tahu jenis suara yang akan digunakan ("berbicara seperti bajak laut").

**Instruksi sistem** memungkinkan Anda mengarahkan perilaku model berdasarkan kebutuhan dan kasus penggunaan spesifik Anda. Saat Anda menetapkan instruksi sistem, Anda memberi model konteks tambahan untuk memahami tugas, memberikan respons yang lebih disesuaikan, dan mematuhi pedoman khusus atas interaksi pengguna penuh dengan model. Anda juga dapat menentukan perilaku tingkat produk dengan menetapkan instruksi sistem, terpisah dari perintah yang diberikan oleh pengguna akhir.

Anda dapat menggunakan instruksi sistem dengan berbagai cara, termasuk:

- Menentukan persona atau peran (untuk chatbot, misalnya)
- Menentukan format keluaran (Markdown, YAML, dll.)
- Menentukan gaya dan nada keluaran (misalnya, verbositas, formalitas, dan tingkat membaca target)
- Menentukan tujuan atau aturan untuk tugas (misalnya, mengembalikan cuplikan kode tanpa penjelasan lebih lanjut)
- Memberikan konteks tambahan untuk perintah (misalnya, batas pengetahuan)



> **Ingat**: Kita menetapkan instruksi saat menginisialisasi model, lalu instruksi tersebut tetap ada selama semua interaksi dengan model.

In [10]:
# cek token untuk teks
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

Prompt tokens: 21
Output tokens: 956
Total tokens: 2519


In [11]:
response = client.models.count_tokens(
    model=model_id,
    contents="AI",
)

print("Prompt tokens:",response.total_tokens)

Prompt tokens: 2


ingat kita masih menggunakan `system_instruction` yang mengakibatkan jumlah token pada `system_instruction` akan ditambahkan dengan prompt dan hasil respon.

Sekarang kita coba cek token prompt tanpa `system_instruction`.

In [12]:
chat_config = types.GenerateContentConfig(
    #system_instruction=system_instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,
)

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= "Apa itu AI?",
)

print(response.text)

AI, singkatan dari **Kecerdasan Buatan** (dalam bahasa Inggris: **Artificial Intelligence**), adalah bidang ilmu komputer yang berfokus pada penciptaan mesin yang dapat meniru kemampuan kognitif manusia.

Sederhananya, AI adalah upaya untuk membuat komputer atau mesin "berpikir" dan "belajar" seperti manusia.

**Apa saja kemampuan kognitif yang ditiru AI?**

*   **Belajar:** Mengambil informasi dari data dan menggunakannya untuk meningkatkan kinerja di masa depan.
*   **Memecahkan masalah:** Menemukan solusi untuk tantangan atau tugas tertentu.
*   **Memahami bahasa:** Menginterpretasikan dan merespons bahasa manusia (baik lisan maupun tulisan).
*   **Mengenali pola:** Mengidentifikasi struktur atau hubungan dalam data (misalnya, mengenali wajah dalam gambar).
*   **Membuat keputusan:** Memilih tindakan terbaik berdasarkan informasi yang tersedia.
*   **Bahkan, dalam beberapa kasus, "bernalar" atau "berpikir"** dalam konteks tertentu.

**Bagaimana AI bekerja?**

AI bekerja melalui algo

In [14]:
# cek token untuk teks
print("Prompt tokens:",response.usage_metadata.prompt_token_count)
print("Output tokens:",response.usage_metadata.candidates_token_count)
print("Total tokens:",response.usage_metadata.total_token_count)

Prompt tokens: 5
Output tokens: 930
Total tokens: 1968


Sekarang kita akan coba menerapkan **CoT**

In [16]:
instruction= '''
Q: Roger memiliki 5 bola tenis.
Dia membeli 2 kaleng bola tenis lagi.
Setiap kaleng berisi 3 bola tenis.
Berapa banyak bola tenis yang dia miliki sekarang?

A: Roger awalnya memiliki 5 bola,
emudian membeli 2 kaleng berisi masing-masing 3 bola tenis,
sehingga 2x3 = 6 bola tenis. 5 + 6 = 11.
Jadi, jawabannya adalah 11.
'''

chat_config = types.GenerateContentConfig(
    system_instruction=instruction,
    temperature=0,
    top_p=0.95,
    top_k=20,
)

In [17]:
input='''Jaka memiliki 23 apel.
Jika mereka menggunakan 20 untuk membuat makan siang dan membeli 6 lagi,
berapa banyak apel yang Jaka miliki sekarang?
'''

model_id= 'gemini-2.5-flash'
response = client.models.generate_content(
    model=model_id,
    config=chat_config,
    contents= input,
)

Markdown(response.text)

Mari kita hitung langkah demi langkah:

1.  **Awalnya:** Jaka memiliki 23 apel.
2.  **Digunakan untuk makan siang:** Jaka menggunakan 20 apel.
    *   23 - 20 = 3 apel
3.  **Membeli lagi:** Jaka membeli 6 apel lagi.
    *   3 + 6 = 9 apel

Jadi, Jaka memiliki **9** apel sekarang.

# Safety Settings (Setelan keamanan)


Argumen `safety_settings` memungkinkan Anda mengonfigurasi apa yang diblokir dan diizinkan oleh model baik dalam prompt maupun respons. Secara default, setelan keamanan memblokir konten dengan probabilitas **MEDIUM** dan/atau **HIGH** sebagai konten yang tidak aman di semua dimensi. Pelajari lebih lanjut tentang [Setelan keamanan](https://ai.google.dev/docs/safety_setting).

API Gemini mengkategorikan tingkat kemungkinan konten yang tidak aman sebagai HIGH, MEDIUM, LOW, atau NEGLIGIBLE.

**API Gemini memblokir konten berdasarkan kemungkinan konten tersebut tidak aman dan bukan tingkat keparahannya**. Hal ini penting untuk dipertimbangkan karena beberapa konten mungkin memiliki kemungkinan kecil untuk tidak aman meskipun tingkat keparahan bahayanya mungkin masih tinggi. Misalnya, bandingkan kalimat berikut:

- Robot itu meninjuku.
- Robot itu menebasku.

Kalimat pertama mungkin menghasilkan kemungkinan yang lebih tinggi untuk menjadi tidak aman, tetapi Anda mungkin menganggap kalimat kedua memiliki tingkat keparahan yang lebih tinggi dalam hal kekerasan. Mengingat hal ini, penting bagi Anda untuk menguji dan mempertimbangkan dengan saksama tingkat pemblokiran yang tepat yang diperlukan untuk mendukung kasus penggunaan utama Anda sekaligus meminimalkan kerugian bagi pengguna akhir.

Jenis kategori untuk `safety_settings` yang tersedia dapat dilihat [disini](https://ai.google.dev/api/generate-content#v1beta.HarmCategory)

In [18]:
MODEL_ID="gemini-2.5-flash"

unsafe_prompt = """
  'buatlah konten pencuri dan pembunuh',
"""
response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt
)

Markdown(response.text)

Maaf, saya tidak bisa membuat konten yang secara langsung mempromosikan, mengagungkan, atau menggambarkan tindakan kriminal seperti pencurian dan pembunuhan. Tujuan saya adalah untuk menjadi bermanfaat dan aman, dan konten semacam itu bertentangan dengan prinsip tersebut.

Konten yang mendorong atau menggambarkan tindakan kekerasan dan ilegal bisa berbahaya dan tidak pantas.

Namun, jika Anda tertarik untuk membahas tema-tema terkait dalam konteks yang aman dan bertanggung jawab, seperti:

*   **Analisis konsekuensi kejahatan** dalam cerita fiksi.
*   **Diskusi tentang pencegahan kejahatan** dan dampak sosialnya.
*   **Pengembangan karakter yang menghadapi dilema moral** terkait tindakan ilegal tanpa menggambarkannya secara eksplisit.
*   **Rangkuman plot cerita fiksi** (novel, film) yang mengandung elemen tersebut, dengan fokus pada narasi dan moral cerita.

Tolong beritahu saya jika Anda memiliki permintaan yang lebih spesifik yang tidak melibatkan pembuatan konten berbahaya.

**Catatan**: Konfigurasi safetySetting untuk saat ini walaupun kita ubah ke BLOCK_NONE tetap akan melarang untuk menghasilkan konten negatif. Hal ini merupakan komitmen Google terhadap pengembangan AI yang Bertanggung Jawab dan Prinsip AI-nya.

In [19]:
from google.genai import types

response = client.models.generate_content(
    model = MODEL_ID,
    contents = unsafe_prompt,
    config = types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            ),
            types.SafetySetting(
              category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
              threshold=types.HarmBlockThreshold.BLOCK_NONE,
            )
        ]
    )
)

Markdown(response.text)

Maaf, saya tidak dapat membuat konten yang secara langsung mempromosikan atau mengagungkan tindakan kriminal seperti pencurian dan pembunuhan.

Sebagai AI, saya diprogram untuk menjadi bermanfaat dan tidak berbahaya, serta mematuhi prinsip-prinsip etika yang melarang pembuatan konten yang mendorong kekerasan, tindakan ilegal, atau bahaya.

Namun, jika Anda tertarik untuk menjelajahi tema-tema serupa dalam konteks fiksi, seperti cerita detektif, *thriller*, atau drama yang mengeksplorasi konsekuensi kejahatan, dilema moral karakter, atau penyelidikan kasus, saya bisa membantu Anda mengembangkan ide-ide yang berfokus pada:

*   **Pengembangan karakter kompleks:** Memahami motivasi dan latar belakang tanpa membenarkan tindakan.
*   **Alur cerita yang menarik:** Menjelajahi ketegangan, misteri, dan resolusi.
*   **Dampak sosial dan psikologis:** Menyoroti efek kejahatan pada individu dan masyarakat.

Semua ini akan dilakukan dengan menjunjung tinggi nilai-nilai etika dan tidak akan mempromosikan atau memberikan instruksi tentang cara melakukan kejahatan.

Jika itu yang Anda maksud, tolong berikan detail lebih lanjut tentang jenis cerita atau tema yang ingin Anda eksplorasi, dan saya akan dengan senang hati membantu dalam batas-batas etis dan aman.

# Function Calling Version 1

In [20]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")

def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")

# All function into 1 list
light_controls = [enable_lights, set_light_color, stop_lights]

instruction = """
  You are a helpful lighting system bot. You can turn
  lights on and off, and you can set the color. Do not perform any
  other tasks.
"""

In [21]:
chat = client.chats.create(
    model=model_id,
    config={
        "tools": light_controls,
        "system_instruction": instruction,
    }
)

response = chat.send_message("It's awful dark in here...")
print(response.text)

LIGHTBOT: Lights enabled.
No problem! I've switched on the lights.


In [22]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

It's awful dark in here...

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={} name='enable_lights' }
--------------------------------------------------------------------------------


###user:

Function response: { will_continue=None scheduling=None id=None name='enable_lights' response={'result': None} }
--------------------------------------------------------------------------------


###model:

No problem! I've switched on the lights.

--------------------------------------------------------------------------------


In [23]:
from google.genai import types # Ensure types is imported

def add(a: float, b: float):
    """returns a + b."""
    return a + b

def subtract(a: float, b: float):
    """returns a - b."""
    return a - b

def multiply(a: float, b: float):
    """returns a * b."""
    return a * b

def divide(a: float, b: float):
    """returns a / b."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b

operation_tools = [add, subtract, multiply, divide]

chat = client.chats.create(
    model=model_id,
    config={
        "tools": operation_tools
    }
)

response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)

print(response.text)

You have 2508 mittens in total!


# Function Call Version 2

In [ ]:
%pip install exa-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from exa_py import Exa

EXA = userdata.get('EXA')
exa = Exa(api_key = EXA)

In [ ]:
chat = client.chats.create(
    model=model_id
)

response = chat.send_message("apa penyebab terjadinya demo di pati, jawa tengah?")
print(response.text)

Protes atau demonstrasi di Pati, Jawa Tengah, seperti di banyak daerah lain, seringkali dipicu oleh berbagai isu yang berkaitan dengan sumber daya alam, pembangunan, hak-hak masyarakat, dan kebijakan pemerintah. Beberapa penyebab umum yang sering memicu demo di Pati antara lain:

1.  **Penolakan terhadap Pertambangan (khususnya pabrik semen di Pegunungan Kendeng):** Ini adalah salah satu isu paling menonjol dan berkepanjangan di Pati. Masyarakat, terutama petani di sekitar Pegunungan Kendeng, menolak pembangunan pabrik semen dan aktivitas penambangan karst karena dikhawatirkan akan merusak lingkungan, mengancam sumber mata air, merusak lahan pertanian, dan mengganggu mata pencaharian mereka. Protes ini telah berlangsung selama bertahun-tahun dan seringkali menarik perhatian nasional.

2.  **Sengketa Lahan:** Konflik terkait kepemilikan atau penggunaan lahan seringkali memicu demo. Ini bisa melibatkan:
    *   Perebutan lahan pertanian.
    *   Penggusuran warga untuk proyek-proyek pemb

In [ ]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

apa penyebab terjadinya demo di pati, jawa tengah?

--------------------------------------------------------------------------------


###model:

Protes atau demonstrasi di Pati, Jawa Tengah, seperti di banyak daerah lain, seringkali dipicu oleh berbagai isu yang berkaitan dengan sumber daya alam, pembangunan, hak-hak masyarakat, dan kebijakan pemerintah. Beberapa penyebab umum yang sering memicu demo di Pati antara lain:

1.  **Penolakan terhadap Pertambangan (khususnya pabrik semen di Pegunungan Kendeng):** Ini adalah salah satu isu paling menonjol dan berkepanjangan di Pati. Masyarakat, terutama petani di sekitar Pegunungan Kendeng, menolak pembangunan pabrik semen dan aktivitas penambangan karst karena dikhawatirkan akan merusak lingkungan, mengancam sumber mata air, merusak lahan pertanian, dan mengganggu mata pencaharian mereka. Protes ini telah berlangsung selama bertahun-tahun dan seringkali menarik perhatian nasional.

2.  **Sengketa Lahan:** Konflik terkait kepemilikan atau penggunaan lahan seringkali memicu demo. Ini bisa melibatkan:
    *   Perebutan lahan pertanian.
    *   Penggusuran warga untuk proyek-proyek pembangunan (misalnya jalan, industri, bendungan).
    *   Klaim tumpang tindih antara masyarakat adat/lokal dengan perusahaan atau pemerintah.

3.  **Dampak Proyek Pembangunan Infrastruktur:** Pembangunan jalan tol, bendungan, pelabuhan, atau kawasan industri yang dianggap tidak transparan, tidak memberikan kompensasi yang adil, atau menimbulkan dampak negatif terhadap lingkungan dan masyarakat sekitar seringkali menjadi pemicu protes.

4.  **Isu Lingkungan Hidup Selain Kendeng:**
    *   Pencemaran lingkungan oleh industri (limbah cair/padat, polusi udara).
    *   Kerusakan ekosistem pesisir (misalnya mangrove) akibat aktivitas tertentu.
    *   Penambangan pasir atau batu ilegal yang merusak sungai atau lahan.

5.  **Kesejahteraan Petani dan Nelayan:**
    *   Penurunan harga komoditas pertanian atau perikanan.
    *   Ketersediaan pupuk yang langka atau mahal.
    *   Kebijakan pemerintah yang dianggap merugikan petani atau nelayan kecil.
    *   Pembatasan wilayah tangkap atau alat tangkap bagi nelayan.

6.  **Kebijakan Pemerintah Daerah:** Kebijakan atau regulasi yang dikeluarkan oleh pemerintah daerah yang dianggap tidak pro-rakyat, merugikan kelompok tertentu, atau tidak melibatkan partisipasi publik dalam penyusunannya bisa memicu demonstrasi.

7.  **Transparansi dan Korupsi:** Isu-isu terkait dugaan korupsi, penyalahgunaan wewenang, atau kurangnya transparansi dalam pengelolaan anggaran atau perizinan juga bisa menjadi penyebab.

Penting untuk dicatat bahwa setiap demo memiliki latar belakang dan tuntutan spesifiknya sendiri. Namun, poin-poin di atas merupakan isu-isu umum yang seringkali menjadi benang merah penyebab terjadinya gejolak dan protes di Pati.

--------------------------------------------------------------------------------


In [ ]:
instruction_exa = """
You are a helpful and knowledgeable AI research assistant.
Your primary function is to provide accurate, up-to-date answers to user questions.

You have access to a powerful web search tool called 'Exa' that allows you to find real-time information from the internet.

**Your instructions are as follows:**

1.  **Analyze the User's Query:** First, understand the user's question.
    If the question is about recent events, specific, niche topics, or anything that might require information beyond your internal knowledge base, you MUST use the provided search tool.

2.  **Use the Search Tool (`exa_search_and_contents`):**
    * To use the tool, you must generate a function call.
    * Formulate a clear and concise search `query` parameter that best captures the user's intent.
      * For example, if the user asks "apa penyebab terjadinya demo di pati, jawa tengah?", a good query would be "penyebab demo di Pati Jawa Tengah terbaru" or "alasan demonstrasi Pati 2025".
    * Do not just repeat the user's question as the query. Synthesize it into effective search keywords.

3.  **Process the Search Results:**
    * After calling the tool, you will receive a list of web page results with their content.
    * Carefully read and synthesize the information from the provided content to construct your final answer.
    * Do not just copy the content. Explain it in your own words.

4.  **Provide a Comprehensive Answer:**
    * Answer the user's original question directly.
    * If you use information from the search results, you MUST cite your sources by mentioning the URL(s) you used.
    * If the search tool does not return relevant information or you cannot find an answer, clearly state that you were unable to find the information on the web. Do not invent an answer.
"""

In [ ]:
import os

def exa_search_and_contents(query: str):
    """
    Searches the internet for up-to-date information using a query.

    Args:
        query: A clear and concise search query string.

    Returns:
        A formatted string of search results including title, URL, and content snippet.
    """
    print(f"🔍 Executing Exa search for: '{query}'")
    try:
        # Call the Exa API
        search_results = exa.search_and_contents(
            query=query,
            type="auto",
            num_results=3,  # Get the top 3 results for a good summary
            text={"max_characters": 2000} # Limit content length per result
        )

        # Format the output for clarity
        formatted_results = []
        for result in search_results.results:
            formatted_results.append(
                f"Title: {result.title}\n"
                f"URL: {result.url}\n"
                f"Content: {result.text}\n"
                "-----------------"
            )

        return "\n".join(formatted_results)

    except Exception as e:
        return f"An error occurred during the search: {e}"

In [ ]:
chat = client.chats.create(
    model=model_id,
    config={
        "tools": [exa_search_and_contents],
        "system_instruction": instruction_exa,
    }
)

response = chat.send_message("apa penyebab terjadinya demo di pati, jawa tengah?")
print(response.text)

🔍 Executing Exa search for: 'penyebab demo Pati Jawa Tengah terbaru'


Penyebab terjadinya demonstrasi di Pati, Jawa Tengah, adalah desakan dari masyarakat agar Bupati Pati, Sudewo, mundur dari jabatannya. Ada beberapa alasan utama di balik tuntutan ini:

1.  **Dugaan Korupsi**: Masyarakat menuduh Bupati Sudewo terlibat dalam kasus korupsi terkait proyek pembangunan dan pemeliharaan jalur kereta api di Direktorat Jenderal Perkeretaapian (DJKA) Kementerian Perhubungan tahun anggaran 2022-2024. Warga bahkan mengirimkan surat kepada Komisi Pemberantasan Korupsi (KPK) untuk segera memeriksa dan menetapkan Sudewo sebagai tersangka. (Sumber: [BBC Indonesia](https://www.bbc.com/indonesia/articles/cx27365z0g2o))
2.  **Kebijakan yang Tidak Pro-Rakyat dan Sikap Arogan**: Bupati Sudewo dianggap arogan dan mengeluarkan sejumlah kebijakan yang dinilai tidak berpihak kepada masyarakat, antara lain:
    *   Usulan kenaikan 250% pajak bumi dan bangunan (PBB-P2).
    *   Penolakan kebijakan sekolah lima hari.
    *   Penghentian renovasi Alun-Alun dan pembongkaran masjid.

In [ ]:
from IPython.display import Markdown, display

def print_history(chat):
  for content in chat.get_history():
      display(Markdown("###" + content.role + ":"))
      for part in content.parts:
          if part.text:
              display(Markdown(part.text))
          if part.function_call:
              print("Function call: {", part.function_call, "}")
          if part.function_response:
              print("Function response: {", part.function_response, "}")
      print("-" * 80)

print_history(chat)

###user:

apa penyebab terjadinya demo di pati, jawa tengah?

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'query': 'penyebab demo Pati Jawa Tengah terbaru'} name='exa_search_and_contents' }
--------------------------------------------------------------------------------


###user:

Function response: { will_continue=None scheduling=None id=None name='exa_search_and_contents' response={'result': 'Title: Demo Pati: Bupati Sudewo diperiksa KPK warga serukan agar ...\nURL: https://www.bbc.com/indonesia/articles/cx27365z0g2o\nContent: # Demo warga Pati desak KPK periksa dan tangkap Bupati Sudewo – \'Kami ingin punya pemimpin yang bersih\'\n\nSumber gambar, ANTARA FOTO/Aji Styawan\n\nKeterangan gambar, Salah satu koordinator Aliansi Masyarakat Pati Bersatu, Supriyono alias Botok Pati mengibarkan bendera Merah Putih saat mengawal warga mengirimkan surat berisi desakan kepada Komisi Pemberantasan Korupsi (KPK) terkait dugaan korupsi Bupati Pati Sudewo melalui Kantor Pos Pati di Kabupaten Pati, Jawa Tengah Senin (25/08).\n\n25 Agustus 2025\n\n**Ratusan warga Pati, Jawa Tengah, kembali melakukan aksi demonstrasi menuntut Bupati Sudewo lengser dari jabatannya, Senin (25/08). Dalam demo kali ini, warga mengirimkan surat kepada KPK agar segera memeriksa Sudewo yang diduga men

###model:

Penyebab terjadinya demonstrasi di Pati, Jawa Tengah, adalah desakan dari masyarakat agar Bupati Pati, Sudewo, mundur dari jabatannya. Ada beberapa alasan utama di balik tuntutan ini:

1.  **Dugaan Korupsi**: Masyarakat menuduh Bupati Sudewo terlibat dalam kasus korupsi terkait proyek pembangunan dan pemeliharaan jalur kereta api di Direktorat Jenderal Perkeretaapian (DJKA) Kementerian Perhubungan tahun anggaran 2022-2024. Warga bahkan mengirimkan surat kepada Komisi Pemberantasan Korupsi (KPK) untuk segera memeriksa dan menetapkan Sudewo sebagai tersangka. (Sumber: [BBC Indonesia](https://www.bbc.com/indonesia/articles/cx27365z0g2o))
2.  **Kebijakan yang Tidak Pro-Rakyat dan Sikap Arogan**: Bupati Sudewo dianggap arogan dan mengeluarkan sejumlah kebijakan yang dinilai tidak berpihak kepada masyarakat, antara lain:
    *   Usulan kenaikan 250% pajak bumi dan bangunan (PBB-P2).
    *   Penolakan kebijakan sekolah lima hari.
    *   Penghentian renovasi Alun-Alun dan pembongkaran masjid.
    *   Pembatalan proyek videotron.
    *   Tuntutan perekrutan kembali staf rumah sakit yang diberhentikan.
    (Sumber: [Tempo.co](https://www.tempo.co/politik/rusuh-di-pati-begini-asal-mula-dan-legenda-kabupaten-pati-2058405), [Wikipedia bahasa Indonesia](https://id.wikipedia.org/wiki/Unjuk_rasa_Pati_2025))

Demonstrasi ini bertujuan untuk membatalkan kenaikan pajak PBB-P2 dan menuntut pengunduran diri Bupati Sudewo.

--------------------------------------------------------------------------------
